In [3]:
import os
from pathlib import Path
from byaldi import RAGMultiModalModel
import torch
import csv
colpali_output = "colpali_output.csv"
dir_path = "papers/"
questions = "question_examples.csv"

In [19]:
def train_colpali():
    model = RAGMultiModalModel.from_pretrained("vidore/colpali")
    index_name = "techpapers"
    
    model.index(input_path=Path(dir_path),
        index_name=index_name,
        store_collection_with_index=True,
        overwrite=True
    )

    return model

async def ask_query(query):
    colpali_page = colpali.search(query, k=1)[0]
    return colpali_page.doc_id, colpali_page.page_num, colpali_page.score, colpali_page.base64

def append_colpali_output(doc_id, document, page_num, score, question, base64):
    isfile = os.path.isfile(colpali_output)
    with open(colpali_output, mode='a', newline='') as file:
        writer = csv.writer(file)
    
        if not isfile:
            writer.writerow(["doc_id", "document", "page_num", "score", "question", "base64"])
        
        writer.writerow([doc_id, document, page_num, score, question, base64])

In [6]:
colpali = train_colpali()

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Indexing file: papers/Human-Object Interaction Detection Collaborated.pdf
Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Index exported to .byal

In [22]:
with open(questions, mode='r', newline='') as file:
    reader = csv.reader(file)
    
    header = next(reader)

    for row_num, row in enumerate(reader, start=1):
        question, document = row
        doc_id, page_num, score, base64 = await ask_query(question)
        append_colpali_output(doc_id, document, page_num, score, question, base64)